# Figure 2
Jupyter notebook preparing the data used for creating the phylogenetic tree with gene heatmap.

In [19]:
import pickle
import math
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import colorcet as cc
import seaborn as sns
from scipy.cluster.hierarchy import linkage, leaves_list, dendrogram, fcluster
from scipy.spatial.distance import pdist
from scipy.stats import chi2_contingency
from sklearn.cluster import AgglomerativeClustering
from collections import Counter

pd.set_option('display.max_columns', None)

In [20]:
genetobb_filename = "../../output/results/v9/no_merge_paralogs/intermediates/panaroo_groupID_to_BBgene.tsv"
bbcoords_filename = "../../ref/B31/bb_rs_gene_coords_b31_v1.tsv"
pa_filename = "../../output/results/v9/no_merge_paralogs/gene_presence_absence.Rtab"
lipoprotein_file = "../0_Data/2_Processed/lipoproteins.csv" # generated in `lipoproteins.csv`
clusterdb_filename = "../../output/results/v9/no_merge_paralogs/intermediates/clustered_proteins_db_results.pkl"
plasmidcolors_filename = "../0_Data/1_Intermediate/plasmid_color_scheme.pkl" # generated in `Figure1a.ipynb`

In [21]:
# pull the best match to pair gene groups to BB gene names
bb = pd.read_csv(genetobb_filename, sep="\t")
bb = bb.sort_values(by=["group", "percent_ident", "alignment_length"], ascending=[True, False, False]).reset_index(drop=True)
bb = bb[["group", "gene"]].drop_duplicates(keep="first").reset_index(drop=True)
bb['count'] = bb.groupby('group').cumcount() + 1
bb = bb[bb["count"]==1].drop(columns="count").reset_index(drop=True)
bb

,group,gene
0,aNKYR,BB_RS01975
1,ackA,BB_RS03145
2,acpP,BB_RS03560
3,acpS,BB_RS00050
4,acrA,BB_RS00695
...,...,...
1527,ysxB,BB_RS03950
1528,yuiD,BB_RS00345
1529,yvaK,BB_RS03275
1530,zupT,BB_RS01085


In [22]:
# import and process coordinates along the genome
coords = pd.read_csv(bbcoords_filename, sep="\t")
coords = coords.drop_duplicates().reset_index(drop=True)
coords['gene'] = coords['gene'].str.replace('gene-', '', regex=False)

bb_coords = bb.merge(coords, on="gene", how="left")
bb_coords_dict = dict(zip(bb_coords["group"], bb_coords["start"]))
bb_coords_dict


{'aNKYR': 410787,
 'ackA': 649170,
 'acpP': 740446,
 'acpS': 10203,
 'acrA': 141979,
 'acyP': 46,
 'ade': 10301,
 'adeC': 22678,
 'adk': 427987,
 'alaS': 224826,
 'alr': 160090,
 'amiC': 703961,
 'ampS': 64038,
 'amrB': 358458,
 'apeA': 374277,
 'apeB': 657546,
 'apt': 818018,
 'arcA': 899499,
 'argF': 900799,
 'argS': 614014,
 'asnS': 98817,
 'aspS': 465518,
 'atpA': 91137,
 'atpB': 89811,
 'atpD': 89200,
 'atpI': 87377,
 'bamA': 836061,
 'bamB': 26399,
 'bamD': 329262,
 'batD': 176069,
 'bcsQ~~~parA': 20766,
 'bdrA': 22131,
 'bdrH': 16497,
 'bdrH~~~repU~~~bdrW': 20043,
 'bdrM': 22102,
 'bdrO': 22113,
 'bdrP': 17089,
 'bdrQ': 22184,
 'bdrR': 16642,
 'bdrW~~~repU': 20043,
 'bepA': 53534,
 'bepA~~~lmp1': 212061,
 'bglX': 768,
 'bmpA': 393044,
 'bmpB': 391932,
 'bmpC': 394105,
 'bmpD': 395481,
 'bosR': 685977,
 'bppB': 24572,
 'bppC': 25175,
 'bptA': 701395,
 'bpuR': 46011,
 'ccmA': 797119,
 'cdaA': 8427,
 'cdd': 645137,
 'cdnL': 365115,
 'cdsA~~~cdsC': 52135,
 'cdsC': 52135,
 'cdsE': 29

In [23]:
# import and process gene presence / absence data
genes = pd.read_csv(pa_filename, sep="\t").set_index("Gene").T.rename(columns={"index":"Isolate","Transporter, dicarboxylate/amino acid:cation (Na+ or H+) symporter (Daacs) family": "Transporter, dicarboxylate/amino acid"})
gene_freqs = genes.sum().to_dict()

print(genes.shape)
genes.head()

(82, 1987)


Gene     group_1286  group_1204  group_1203  group_1202  group_1201  \
B331P             1           1           1           1           1   
B418P             1           1           1           1           1   
B500P             1           1           1           1           1   
ESI26H            1           1           1           1           1   
ESI361H           1           1           1           1           1   

Gene     group_1200  group_1199  nCS2  guaB  immA  acyP  celA  group_1196  \
B331P             1           1     1     1     1     1     1           1   
B418P             1           1     1     1     1     1     1           1   
B500P             1           1     1     1     1     1     1           1   
ESI26H            1           1     1     1     1     1     1           1   
ESI361H           1           1     1     1     1     1     1           1   

Gene     group_1195  group_1194  group_1193  celC  celB  resT  guaA  \
B331P             1           1           1     1     1     1     1   
B418P             1           1           1     1     1     1     1   
B500P             1           1           1     1     1     1     1   
ESI26H            1           1           1     1     1     1     1   
ESI361H           1           1           1     1     1     1     1   

Gene     group_1192  group_1191  group_1190  group_1189  group_1188  pspE  \
B331P             1           1           1           1           1     1   
B418P             1           1           1           1           1     1   
B500P             1           1           1           1           1     1   
ESI26H            1           1           1           1           1     1   
ESI361H           1           1           1           1           1     1   

Gene     tmk  tadD  group_1186  group_1183  wcaA  group_1179  pepF  pdeB  \
B331P      1     1           1           1     1           1     1     1   
B418P      1     1           1           1     1           1     1     1   
B500P      1     1           1           1     1           1     1     1   
ESI26H     1     1           1           1     1           1     1     1   
ESI361H    1     1           1           1     1           1     1     1   

Gene     folD  yliI  yqxC  potA  group_1163  group_1161  pepP  group_1158  \
B331P       1     1     1     1           1           1     1           1   
B418P       1     1     1     1           1           1     1           1   
B500P       1     1     1     1           1           1     1           1   
ESI26H      1     1     1     1           1           1     1           1   
ESI361H     1     1     1     1           1           1     1           1   

Gene     corB~~~tlyC  hMG1  pASTA  group_1156  group_1155  tolC  group_1153  \
B331P              1     1      1           1           1     1           1   
B418P              1     1      1           1           1     1           1   
B500P              1     1      1           1           1     1           1   
ESI26H             1     1      1           1           1     1           1   
ESI361H            1     1      1           1           1     1           1   

Gene     group_1152  yfmG  yitT  lepA  group_1151  batD  mutL  yibQ  queA  \
B331P             1     1     1     1           1     1     1     1     1   
B418P             1     1     1     1           1     1     1     1     1   
B500P             1     1     1     1           1     1     1     1     1   
ESI26H            1     1     1     1           1     1     1     1     1   
ESI361H           1     1     1     1           1     1     1     1     1   

Gene     group_1147  rlmB  group_1143  group_1142  hisS  group_1141  \
B331P             1     1           1           1     1           1   
B418P             1     1           1           1     1           1   
B500P             1     1           1           1     1           1   
ESI26H            1     1           1           1     1           1   
ESI361H

In [24]:
# import raw data with gene presence on plasmids / isolates
gene_dict = pd.read_pickle(clusterdb_filename)

# initialize an empty list to store the processed data
processed_data = []

# iterate over the dictionary
for gene, df in gene_dict.items():
    # add the gene name to the dataframe
    df['gene'] = gene
    # append the dataframe to the list
    processed_data.append(df)

# concatenate all the dataframes in the list into a single dataframe
gene_tree_df = pd.concat(processed_data)
gene_tree_df["replicon_name"] = gene_tree_df["replicon_name"].replace("NA", "Unclassified")
gene_tree_df['assembly'] = gene_tree_df['assembly'].str.replace(r'GCF_\d+\.\d+_', '', regex=True)  # remove GCF accession with underscore
gene_tree_df['assembly'] = gene_tree_df['assembly'].str.replace(r'_genomic', '', regex=True)  # remove "_genomic"
print(gene_tree_df.shape)

(109623, 30)


In [25]:
# check the number of gene groups encoded on each replicon
gene_tree_df["replicon_name"].value_counts()

replicon_name
chromosome      66891
lp54             5177
cp32-7           2155
cp26             2139
cp32-9           2127
cp32-6           2080
lp36             2036
lp38             1960
lp56             1775
cp32-1+5         1743
cp32-3           1702
cp32-11          1697
lp28-4           1573
cp32-4           1526
lp28-3           1517
lp25             1399
lp28-1           1375
lp17             1258
lp28-2           1209
cp32-12          1109
cp32-8           1075
lp28-6           1046
cp32-2            992
cp32-1            909
cp32-10           604
cp32-5            502
lp28-5            478
lp21-cp9          350
Unclassified      205
lp28-7            198
cp32-13           193
cp9               183
lp28-8            117
cp32-3+10          99
lp28-9             86
lp21               63
cp9-3              38
lp5                34
lp28-11             2
cp32-9-4            1
Name: count, dtype: int64

In [26]:
# check plasmid presence by isolate
plasmid_presence = gene_tree_df[["assembly", "replicon_name"]].drop_duplicates(keep="first").reset_index(drop=True)
plasmid_presence

,assembly,replicon_name
0,ASM1913465v1,lp54
1,UCT32H,lp54
2,URI111H,lp54
3,UNY193P,lp54
4,UCT113H,lp54
...,...,...
1505,URI102H,cp32-2
1506,URI36H,lp28-5
1507,URI118H,cp32-8
1508,UCT30H,cp32-8


In [27]:
# verify plasmid presence counts across the sample
plasmid_presence["replicon_name"].value_counts()

replicon_name
lp54            82
chromosome      82
cp26            82
lp17            80
lp28-3          71
lp25            70
cp32-7          68
lp36            68
lp28-4          66
lp38            62
cp32-9          58
cp32-6          57
lp28-1          55
cp32-11         55
cp32-3          52
cp32-1+5        51
cp32-4          39
cp32-12         38
lp28-6          37
lp56            37
cp32-8          36
lp28-2          33
Unclassified    32
cp32-1          27
lp28-5          26
cp32-2          26
cp32-5          23
cp9             21
cp32-10         19
lp21-cp9        15
lp28-7           7
lp28-8           7
lp21             6
cp32-13          6
lp5              6
cp9-3            4
lp28-9           2
cp32-3+10        2
cp32-9-4         1
lp28-11          1
Name: count, dtype: int64

In [28]:
ordered_plasmids = [x for x in plasmid_presence["replicon_name"].value_counts().index]
ordered_plasmids[0], ordered_plasmids[1] = ordered_plasmids[1], ordered_plasmids[0]
# load plasmid color-mapping
with open(plasmidcolors_filename, 'rb') as pickle_file:
    plasmid_color_scheme = pickle.load(pickle_file)


In [29]:
# add color assignments to genes based on the encoding replicon
gene_tree_df = gene_tree_df[['assembly', 'gene', 'replicon_name']]
gene_tree_df.reset_index(drop=True, inplace=True)
gene_tree_df["color"] = gene_tree_df["replicon_name"].apply(lambda z: plasmid_color_scheme[z])
print(len(gene_tree_df["gene"].unique()))
gene_tree_df

1987


,assembly,gene,replicon_name,color
0,ASM1913465v1,group_1286,lp54,#018700
1,UCT32H,group_1286,lp54,#018700
2,URI111H,group_1286,lp54,#018700
3,UNY193P,group_1286,lp54,#018700
4,UCT113H,group_1286,lp54,#018700
...,...,...,...,...
109618,URI120H,group_227,cp32-11,#9a6900
109619,URI120H,erpP,cp32-11,#9a6900
109620,URI48H,group_204,cp32-1+5,#fdf490
109621,ASM4079076v1,group_38,lp28-8,#a0e491


In [30]:
# add lipoprotein status to genes
lipo = pd.read_csv(lipoprotein_file)
gene_tree_df = gene_tree_df.merge(lipo, on="gene", how="left")
gene_tree_df

,assembly,gene,replicon_name,color,Localization,lipo,surface_lipo
0,ASM1913465v1,group_1286,lp54,#018700,NaN,Other,Other
1,UCT32H,group_1286,lp54,#018700,NaN,Other,Other
2,URI111H,group_1286,lp54,#018700,NaN,Other,Other
3,UNY193P,group_1286,lp54,#018700,NaN,Other,Other
4,UCT113H,group_1286,lp54,#018700,NaN,Other,Other
...,...,...,...,...,...,...,...
109618,URI120H,group_227,cp32-11,#9a6900,S,Lipoprotein,Surface
109619,URI120H,erpP,cp32-11,#9a6900,S,Lipoprotein,Surface
109620,URI48H,group_204,cp32-1+5,#fdf490,NaN,Other,Other
109621,ASM4079076v1,group_38,lp28-8,#a0e491,NaN,Other,Other


In [31]:
# calculate the number of occurrences of each gene on different plasmids across the dataset
# identifies the plasmid that each gene is most commonly located on
replicon_counts = gene_tree_df.groupby(['gene', 'replicon_name']).size().reset_index(name='count')

best_replicon = replicon_counts.loc[replicon_counts.groupby('gene')['count'].idxmax()]
best_replicon_dict = dict(zip(best_replicon["gene"], best_replicon["replicon_name"]))

replicon_counts = replicon_counts.pivot_table(index='gene', columns='replicon_name', values='count', fill_value=0)

replicon_counts = replicon_counts.reset_index()
replicon_counts

replicon_name,gene,Unclassified,chromosome,cp26,cp32-1,cp32-1+5,cp32-10,cp32-11,cp32-12,cp32-13,cp32-2,cp32-3,cp32-3+10,cp32-4,cp32-5,cp32-6,cp32-7,cp32-8,cp32-9,cp32-9-4,cp9,cp9-3,lp17,lp21,lp21-cp9,lp25,lp28-1,lp28-11,lp28-2,lp28-3,lp28-4,lp28-5,lp28-6,lp28-7,lp28-8,lp28-9,lp36,lp38,lp5,lp54,lp56
0,aNKYR,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aac3Ia,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ackA,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,acpP,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,acpS,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,ysxB,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1983,yuiD,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,yvaK,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1985,zupT,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# calculate the most frequent replicon for each gene and sort the dictionary
most_frequent_replicon_series = gene_tree_df.groupby('gene')['replicon_name'].agg(lambda x: x.mode().iloc[0])
replicon_dict = most_frequent_replicon_series.to_dict()
replicon_dict = dict(sorted(
    replicon_dict.items(),
    key=lambda item: ordered_plasmids.index(item[1]) if item[1] in ordered_plasmids else float('inf')
))
with open("../0_Data/1_Intermediate/gene_with_best_replicon_hit.pkl", 'wb') as fp:
    pickle.dump(replicon_dict, fp)

In [33]:
# process the data for tree construction
gene_tree_df["x"]=1
gene_tree_df["x"] = gene_tree_df.groupby("gene")["x"].transform("sum")
gene_tree_df["best_replicon"] = gene_tree_df["gene"].apply(lambda z: best_replicon_dict[z])
gene_tree_df["pos"] = gene_tree_df["gene"].apply(lambda z: bb_coords_dict[z] if z in bb_coords_dict.keys() else 1000000)
gene_tree_df = gene_tree_df.merge(replicon_counts, on="gene", how="left")
gene_tree_df['replicon_name'] = pd.Categorical(gene_tree_df['replicon_name'], categories=ordered_plasmids, ordered=True)
gene_tree_df['best_replicon'] = pd.Categorical(gene_tree_df['best_replicon'], categories=ordered_plasmids, ordered=True)
gene_tree_df = gene_tree_df.sort_values(by=["best_replicon", "pos", "x", "gene"], ascending=[True, True, False, True]).reset_index(drop=True)

# save thresholded gene output
thresh = 26
gene_tree_df_subset = gene_tree_df[gene_tree_df["x"] >= thresh].copy().drop(columns="x")
gene_tree_df_subset.to_csv("../0_Data/2_Processed/GeneTree_Ordered_Thresh.csv",
                    index=False)

# save the full gene output
gene_tree_df["prevalence"] = gene_tree_df["x"] / len(gene_tree_df["assembly"].unique())
gene_tree_df = gene_tree_df.drop(columns="x")
gene_tree_df.to_csv("../0_Data/2_Processed/GeneTree_Ordered.csv",
                    index=False)
gene_tree_df


,assembly,gene,replicon_name,color,Localization,lipo,surface_lipo,best_replicon,pos,Unclassified,chromosome,cp26,cp32-1,cp32-1+5,cp32-10,cp32-11,cp32-12,cp32-13,cp32-2,cp32-3,cp32-3+10,cp32-4,cp32-5,cp32-6,cp32-7,cp32-8,cp32-9,cp32-9-4,cp9,cp9-3,lp17,lp21,lp21-cp9,lp25,lp28-1,lp28-11,lp28-2,lp28-3,lp28-4,lp28-5,lp28-6,lp28-7,lp28-8,lp28-9,lp36,lp38,lp5,lp54,lp56,prevalence
0,ASM1913465v1,group_1150,chromosome,#bceddb,NaN,Lipoprotein,Other,chromosome,105,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963415
1,UCT32H,group_1150,chromosome,#bceddb,NaN,Lipoprotein,Other,chromosome,105,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963415
2,URI111H,group_1150,chromosome,#bceddb,NaN,Lipoprotein,Other,chromosome,105,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963415
3,UCT113H,group_1150,chromosome,#bceddb,NaN,Lipoprotein,Other,chromosome,105,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963415
4,UCT29H,group_1150,chromosome,#bceddb,NaN,Lipoprotein,Other,chromosome,105,0.0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.963415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109618,PFhe_I_PB_Ill_cons,group_864,cp32-3+10,#72b8ff,NaN,Other,Other,cp32-3+10,11924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024390
109619,ESI26H,group_254,cp32-3+10,#72b8ff,NaN,Other,Other,cp32-3+10,22184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024390
109620,PFhe_I_PB_Ill_cons,group_254,cp32-3+10,#72b8ff,NaN,Other,Other,cp32-3+10,22184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024390
109621,ESI26H,group_369,cp32-3+10,#72b8ff,S,Lipoprotein,Surface,cp32-3+10,26257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024390
